# Samokat.tech Matching

## Описание проекта

**Matching** - это задача поиска и сопоставления двух объектов из разных наборов данных. Такая потребность возникла у маркетплейса, который имеет большой перечен товаров для продажи. На склад собираются поступить новые товары, которые могут имеет небольшие различия от уже существующих. Технически, это выглядит как совершенно новые товары. Фактически это те же товары, которые уже продаются на маркетплейсе. Задача сопоставить и связать новые (предлагаемые для продажи) товары со старыми, пользуясь совпадениями в характеристиках, описаниях и изображениях.

**Что надо сделать?**
- предстоит реализовать финальную часть пайплайна матчинга. В ней необходимо принять решение для каждой пары (товар предлагаемый продавцом — товар на площадке), является ли она матчем или нет (бинарная классификация).
- Для этого у каждой пары есть набор признаков и наборы векторов (картиночные и текстовые), которые описывают товары из этой пары.
- В качестве метрики качества решения используется _F-score_.

**Данные:** [источник](https://www.kaggle.com/competitions/binary-classification-offers-on-the-marketplace/data)

- *train.csv -* обучающий датасет. Содержит пары предложений и товаров, вероятность их мэтча, а так же другие параметры (id товаров, категория и др.)
- *test.csv* - датасет с товарами (уникальный **id** и вектор признаков), для которых надо найти наиболее близкие товары из *base.csv*
- *sample_submission.csv* - пример файла предсказаний.

Каждое предложение и товар имеют изображение и название с атрибутами, которые в свою очередь представлены в векторном виде (эмбеддингах).
Embeddings:

- `goods_image_vectors` и `offer_image_vectors` - содержат файлы с векторами изображений (embed_deperson.npy) и их идентификаторами (items_deperson.npy) для товаров ассортимента и предложений соответственно. Объекты в файлах соотносятся 1 к 1.
- `goods_title_vectors` и `offer_title_vectors` - содержат файлы с векторами названий+атрибутов (embed_deperson.npy) и их идентификаторами (items_deperson.npy) для товаров ассортимента и предложений соответственно. Объекты в файлах соотносятся 1 к 1.

**План работы:**
- Загрузить данные
- Понять задачу
- Подготовить данные
- Обучить модель
- Измерить качество
- git commit
- Сделать лучше!

Примечание: ячейки кода, исполнение которы занимает продолжительное время, имеют функцию `%%time` и время исполнения, указанное в конце выходных данных. 

___

## Подготовка

In [1]:
# импорт библиотек
#import phik
#import faiss
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# импорт спец. модулей
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, roc_curve, ConfusionMatrixDisplay

# константы
RANDOM_STATE = 42
DATA_DIR = 'data/'
ITEMS_FILENAME = 'items_deperson.npy'
EMBEDDINGS_FILENAME = 'embed_deperson.npy'

In [2]:
# чтение датасетов
train_df          = pd.read_csv(DATA_DIR + 'train.csv')
test_df           = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

offer_image_embed = np.load(DATA_DIR + 'offer_image_vectors/embed_deperson.npy')
offer_image_items = np.load(DATA_DIR + 'offer_image_vectors/items_deperson.npy')

При первичном знакомстве с данными обнаружил опечатку в названиях пары столбцов. Дабы не множить её, я внесу исправление в самом начале. 

`offer_depersAnalised` -> `offer_depersOnalised`

`goods_depersAnalised` -> `goods_depersOnalised`

In [3]:
# коррекция опечатки
for df in (train_df, test_df):
    df.rename(columns={'offer_depersanalised' : 'offer_depersonalised',
                       'goods_depersanalised' : 'goods_depersonalised'}, inplace=True)
    print('Обновлённые названия:', list(df.columns[0:2]))

Обновлённые названия: ['offer_depersonalised', 'goods_depersonalised']
Обновлённые названия: ['offer_depersonalised', 'goods_depersonalised']


___

## Анализ данных

Описание колонок:

- `offer_depersonalised` и `goods_depersonalised` - идентификаторы предложения и товара соответственно
- `sum_length` - суммарная длина пары названий и атрибутов в символах
- `dist` - расстояние между названиями предложения и товара*
- `attrs+title_score` - вероятность матча от рескоринговой модели
- `offer_price` и `item_price` - цена предложения и товара соответственно
- `goods_category_id` - категория товара
- `id` - идентификатор пары offer_depersonalised + $ + goods_depersonalised
- `target` (только в train.csv) - метка класса (0 - не матч, 1 - матч)

\* столбец заявлен, но отсутствует в исходных данных.

### Train

In [4]:
# вывод общей информации
display(train_df.head())
train_df.info(verbose=True, show_counts=True)
print('Кол-во мэтчей-дубликатов:', train_df.duplicated().sum())
print('Кол-во предложений:', len(train_df['offer_depersonalised'].unique()))
print('Кол-во товаров:', len(train_df['goods_depersonalised'].unique()))

# выделение цел. признака из обучающей выборки
train_targets = train_df['target']
train_df.drop('target', axis=1, inplace=True)

,offer_depersonalised,goods_depersonalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,target,id
0,295140,1396793,37,0.027267,1070,NaN,14.0,0,295140$1396793
1,65291,1396586,38,0.050415,698,NaN,14.0,0,65291$1396586
2,39232,1396244,38,0.087280,837,NaN,14.0,0,39232$1396244
3,39232,1396513,38,0.087280,837,NaN,14.0,0,39232$1396513
4,65052,1396237,38,0.079773,1085,NaN,14.0,0,65052$1396237


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518441 entries, 0 to 2518440
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   offer_depersonalised  2518441 non-null  int64  
 1   goods_depersonalised  2518441 non-null  int64  
 2   sum_length            2518441 non-null  int64  
 3   attrs+title_score     2518441 non-null  float64
 4   offer_price           2518441 non-null  int64  
 5   goods_price           2111154 non-null  float64
 6   goods_category_id     2517608 non-null  float64
 7   target                2518441 non-null  int64  
 8   id                    2518441 non-null  object 
dtypes: float64(3), int64(5), object(1)
memory usage: 172.9+ MB
Кол-во мэтчей-дубликатов: 0
Кол-во предложений: 500000
Кол-во товаров: 1592607


Всего около 2,5 млн пар. Пропуски присутствуют лишь в `goods_price` и `goods_category_id`.

### Test

In [5]:
# вывод общей информации
display(test_df.head())
test_df.info(verbose=True, show_counts=True)
print('Кол-во мэтчей-дубликатов:', test_df.duplicated().sum())
print('Кол-во предложений:', len(test_df['offer_depersonalised'].unique()))
print('Кол-во товаров:', len(test_df['goods_depersonalised'].unique()))

,offer_depersonalised,goods_depersonalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,id
0,64819,1396468,38,0.046997,368,NaN,14.0,64819$1396468
1,64819,1396235,38,0.046997,368,NaN,14.0,64819$1396235
2,64819,1396318,38,0.046997,368,NaN,14.0,64819$1396318
3,359959,1396281,40,0.060211,634,NaN,14.0,359959$1396281
4,142700,717657,40,0.000370,14924,31840.0,2.0,142700$717657


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363835 entries, 0 to 363834
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   offer_depersonalised  363835 non-null  int64  
 1   goods_depersonalised  363835 non-null  int64  
 2   sum_length            363835 non-null  int64  
 3   attrs+title_score     363835 non-null  float64
 4   offer_price           363835 non-null  int64  
 5   goods_price           304864 non-null  float64
 6   goods_category_id     363704 non-null  float64
 7   id                    363835 non-null  object 
dtypes: float64(3), int64(4), object(1)
memory usage: 22.2+ MB
Кол-во мэтчей-дубликатов: 0
Кол-во предложений: 72767
Кол-во товаров: 316987


### Embeddings и массивы

In [6]:
# эмбеддинги
display(offer_image_embed)
print('Размер массива:', offer_image_embed.shape)

max_value = max(max(row) for row in offer_image_embed)
min_value = min(min(row) for row in offer_image_embed)
print('Макс. значение массива:', max_value)
print('Мин. значение массива:', min_value)

array([[ 0.31286708,  0.9922713 ,  1.2100751 , ..., -1.7545763 ,
        -0.23919716, -1.2425919 ],
       [ 2.4316337 ,  0.9014603 , -0.22259077, ..., -1.1001699 ,
        -1.1482008 , -0.18731171],
       [ 1.0660228 , -0.752132  ,  1.1504172 , ...,  0.9137148 ,
        -0.8000038 , -0.83978504],
       ...,
       [ 0.15886657,  1.1132201 ,  1.8915755 , ..., -1.9614205 ,
         0.25737146,  2.2726674 ],
       [-0.07633805,  1.1760142 ,  1.7563714 , ...,  0.8059396 ,
        -0.01547651, -1.1802496 ],
       [ 2.2704175 , -0.9822532 , -0.21690361, ...,  1.8719108 ,
        -0.7007608 , -1.5808858 ]], dtype=float32)

Размер массива: (457586, 256)
Макс. значение массива: 10.499202
Мин. значение массива: -10.556353


In [7]:
# массив id предложений
display(offer_image_items)
print('Размер массива:', offer_image_items.shape)

array(['140', '185', '187', ..., '572691', '572699', '572735'],
      dtype='<U6')

Размер массива: (457586,)
